<a href="https://colab.research.google.com/github/Machine-Learning-Tokyo/tfjs-workshop/blob/master/char-rnn/keras_lstm_text_gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build and train the LSTM.

This script is copied from https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py

In [11]:
'''
#Example script to generate text from Nietzsche's writings.
At least 20 epochs are required before the generated text
starts sounding coherent.
It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.
If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''

from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

path = get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.02, 0.04, 0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=5,
          callbacks=[print_callback])

corpus length: 600893
total chars: 57
nb sequences: 200285
Vectorization...
Build model...
Epoch 1/5
200285/200285 [==============================] - 228s 1ms/step - loss: 1.9739

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "he very nature and being of the
"thing-i"
he very nature and being of the
"thing-interstance of the sense of the spectations and and the stands and all the spection of the such a surding and a conar and the sense and the sense of the sense and the sense and the sense and the conscience of the sense and the sense and all the sense and the sense and the sense and the spected and and the such and the sense and the courder to the constinction, and interpention of a sentices and int
----- diversity: 0.5
----- Generating with seed: "he very nature and being of the
"thing-i"
he very nature and being of the
"thing-inced the possible to a some the the deed to the for the morality of he as nature and procise in the respossice moralit

# Save the Keras model

In [0]:
model.save('char_rnn.h5')

# Build the character to index dictionaries

In [0]:
import json

with open('char_indices.js', 'w') as f:
  char_indices_str = json.dumps(char_indices, sort_keys=True, indent=2)
  char_indices_str = 'export default ' + char_indices_str
  f.write(char_indices_str)
  
with open('indices_char.js', 'w') as f:
  indices_char_str = json.dumps(indices_char, sort_keys=True, indent=2)
  indices_char_str = 'export default ' + indices_char_str
  f.write(indices_char_str)

# Use tfjs-converter to convert the saved Keras model

In [9]:
!pip install tensorflowjs

    100% |████████████████████████████████| 13.9MB 2.4MB/s 
    100% |████████████████████████████████| 84.7MB 394kB/s 
    100% |████████████████████████████████| 61kB 22.8MB/s 
    100% |████████████████████████████████| 3.0MB 9.2MB/s 
    100% |████████████████████████████████| 358kB 22.1MB/s 
featuretools 0.4.1 has requirement pandas>=0.23.0, but you'll have pandas 0.22.0 which is incompatible.
datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
  Found existing installation: numpy 1.14.6
    Uninstalling numpy-1.14.6:
      Successfully uninstalled numpy-1.14.6


In [0]:
!tensorflowjs_converter --input_format=keras char_rnn.h5 char_rnn_tfjs